In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
plt.close('all')

# reading csv file
data18 = pd.read_csv("2018.csv")
data19 = pd.read_csv("2019.csv")

MAX = len(data18.index)
SAMPLE = 150

# Removing the un-needed columns
columns = ['Photo', 'Flag', 'Club Logo']
data18.drop(columns, inplace=True, axis=1)

# Renaming columns to match
data18.rename(columns={'Preferred Positions': 'Position'}, inplace=True)

# Strip trailing whitespace from end of Positions
data18['Position'] = data18['Position'].str.strip()

# print(list(data18.columns.values))

In [8]:
def overall(id = None):
    def compare_overall(row):
        overall2018 = row['Overall']

        new_stats = data19.loc[data19['ID'] == row['ID']]['Overall']
        row = pd.Series({'Name': row['Name'], 'Overall 2018': overall2018})
        if new_stats.size > 0:
            row['Overall 2019'] = new_stats.tolist()[0]
            row['Overall Difference'] = new_stats.tolist()[0] - overall2018
            return row
        else:
            row['Overall 2019'] = np.NaN
            row['Overall Difference'] = np.NaN
            return row

    if id:
        player = data18[data18['ID'] == id]
        result = pd.DataFrame(player.apply(compare_overall, axis=1))
    else:
        result = pd.DataFrame(data18.apply(compare_overall, axis=1))

    return result

print(overall())

                    Name  Overall 2018  Overall 2019  Overall Difference
0      Cristiano Ronaldo            94          94.0                 0.0
1               L. Messi            93          94.0                 1.0
2              Neymar Jr            92          92.0                 0.0
3              L. Suárez            92          91.0                -1.0
4               M. Neuer            92          89.0                -3.0
5         R. Lewandowski            91          90.0                -1.0
6                 De Gea            90          91.0                 1.0
7              E. Hazard            90          91.0                 1.0
8               T. Kroos            90          90.0                 0.0
9             G. Higuaín            90          88.0                -2.0
10          Sergio Ramos            90          91.0                 1.0
11          K. De Bruyne            89          91.0                 2.0
12           T. Courtois            89          89.

In [ ]:
# acceleration agility finishing...

In [9]:
print('In which position are the oldest players (eg. Goalkeepers)?')

oldest18 = data18.loc[data18['Age'] == data18['Age'].max()]
print('Oldest Player of 2018 is ' + oldest18.iloc[0]['Name'] + ' with the age of ' + str(oldest18.iloc[0]['Age']) +
      ' with the position: ' + oldest18.iloc[0]['Position'])

oldest19 = data19.loc[data19['Age'] == data19['Age'].max()]
print('Oldest Player of 2019 is ' + oldest19.iloc[0]['Name'] + ' with the age of ' +
      str(oldest19.iloc[0]['Age']) + ' with the position: ' + oldest19.iloc[0]['Position'])

In which position are the oldest players (eg. Goalkeepers)?
Oldest Player of 2018 is B. Richardson with the age of 47 with the position: GK
Oldest Player of 2019 is O. Pérez with the age of 45 with the position: GK


In [10]:
print('Which players had the most change in value and why? is it based on their overall improvement?')

data19['Value'] = data19['Value'].map(lambda x: x.lstrip('€'))
data18['Value'] = data18['Value'].map(lambda x: x.lstrip('€'))

data18['Value'] = (data18['Value'].replace(r'[KM]+$', '', regex=True).astype(float) *
                   data18['Value'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1)
                   .replace(['K', 'M'], [10 ** 3, 10 ** 6]).astype(int))

data19['Value'] = (data19['Value'].replace(r'[KM]+$', '', regex=True).astype(float) *
                   data19['Value'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1)
                   .replace(['K', 'M'], [10 ** 3, 10 ** 6]).astype(int))

def link_values(row):

    value18 = row[['Value', 'Overall']]

    new_stats = data19.loc[data19['ID'] == row['ID']]
    new_stats = new_stats[['Value', 'Overall']]

    row = pd.Series({'ID': row['ID'], 'Name': row['Name'], 'Value 2018': value18['Value'], 'Overall 2018': value18['Overall']})
    if new_stats.size > 0:
        new_stats = new_stats.values[0]
        row['Value 2019'] = new_stats[0]
        row['Overall 2019'] = new_stats[1]
        row['Overall Difference'] = new_stats[1] - value18['Overall']
        return row
    else:
        row['Value 2019'] = np.NaN
        row['Overall 2019'] = np.NaN
        row['Overall Difference'] = np.NaN
        return row

result = pd.DataFrame(data18.apply(link_values, axis=1))
result = result.dropna(axis='rows')

change = pd.DataFrame({'ID': result['ID'], 'Name': result['Name'], 'Value Change': result['Value 2019'] - result['Value 2018'], 'Overall Change': result['Overall Difference']})

change = change.sort_values(by=['Value Change'], ascending=False)
most_increase = change.head(10)
most_decrease = change.tail(10).iloc[::-1]

print('List of players with the most increase in value: ')
print(most_increase)
print('List of players with the most decrease in value: ')
print(most_decrease)

Which players had the most change in value and why? is it based on their overall improvement?
List of players with the most increase in value: 
          ID                 Name  Value Change  Overall Change
2437  232363          M. Škriniar    39500000.0            11.0
127   231747            K. Mbappé    39500000.0             5.0
137   209331             M. Salah    37000000.0             5.0
393   223848  S. Milinković-Savić    30500000.0             5.0
162   188152                Oscar    30500000.0             0.0
142   205600            S. Umtiti    27500000.0             4.0
229   202652          R. Sterling    27000000.0             4.0
199   222492              L. Sané    26500000.0             4.0
5515  231866              Rodrigo    26300000.0            12.0
246   192387          C. Immobile    26000000.0             5.0
List of players with the most decrease in value: 
        ID               Name  Value Change  Overall Change
13  184941         A. Sánchez   -30000000.

In [19]:
print('Is the overall improvement based on age?')
def link_values(row):

    value18 = row['Overall']

    new_stats = data19.loc[data19['ID'] == row['ID']]
    new_stats = new_stats[['Age', 'Overall']]

    row = pd.Series({'Name': row['Name']})
    if new_stats.size > 0:
        new_stats = new_stats.values[0]
        row['Age'] = new_stats[0]
        row['Overall Difference'] = new_stats[1] - value18
        return row
    else:
        row['Age'] = np.NaN
        row['Overall Difference'] = np.NaN
        return row

%timeit result = pd.DataFrame(data18.apply(link_values, axis=1))
result = result.dropna(axis='rows')

result = result.sort_values(by=['Overall Difference'], ascending=False)
highest_overall = result.head(10)

print('List of players with the most overall rating along with their age: ')
print(highest_overall)

Is the overall improvement based on age?
51.2 s ± 406 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
List of players with the most overall rating along with their age: 
                 Name   Age  Overall Difference
17448      S. Cáseres  21.0                20.0
16977  A. Wan-Bissaka  20.0                19.0
17491       M. Turner  24.0                17.0
17445     E. Brignola  18.0                17.0
16063  T. Koopmeiners  20.0                17.0
17260          E. Eze  20.0                16.0
12955     João Novais  24.0                16.0
14450        J. Firpo  21.0                16.0
12828     M. Dúbravka  29.0                15.0
16771      M. Varnier  20.0                15.0


In [13]:
print('Which nationality has the best overall average?')

def link_values(row):

    value18 = row[['Nationality', 'Overall']]

    new_stats = data19.loc[data19['ID'] == row['ID']]['Overall']

    row = pd.Series({'Nationality': row['Nationality'], 'Overall 2018': value18['Overall']})
    if new_stats.size > 0:
        new_stats = new_stats.values
        row['Overall 2019'] = new_stats[0]
        row['Overall Difference'] = new_stats[0] - value18['Overall']
        return row
    else:
        row['Overall 2019'] = np.NaN
        row['Overall Difference'] = np.NaN
        return row

result = pd.DataFrame(data18.apply(link_values, axis=1))
result = result.dropna(axis='rows')

result = result.groupby('Nationality').mean()
result = result.sort_values(by=['Overall Difference'], ascending=False)

print(result)

Which nationality has the best overall average?
                      Overall 2018  Overall 2019  Overall Difference
Nationality                                                         
Dominican Republic       68.000000     78.000000           10.000000
Afghanistan              55.000000     61.000000            6.000000
Philippines              68.000000     73.000000            5.000000
Palestine                64.000000     69.000000            5.000000
Hong Kong                54.000000     59.000000            5.000000
Antigua & Barbuda        59.333333     64.000000            4.666667
Sudan                    60.000000     64.000000            4.000000
Mauritania               63.000000     67.000000            4.000000
Zambia                   64.000000     67.666667            3.666667
Iran                     68.750000     71.500000            2.750000
Burkina Faso             67.100000     69.800000            2.700000
Gambia                   66.444444     69.111111       

In [ ]:
print('Is the potential of the 2018 dataset correspond to the overall of the 2019 dataset?')

def link_values(row):

    value18 = row['Potential']

    new_stats = data19.loc[data19['ID'] == row['ID']]['Overall']

    row = pd.Series({'Name': row['Name']})
    if new_stats.size > 0:
        new_stats = new_stats.values
        row['Potential was correct'] = value18 == new_stats[0]
        return row
    else:
        row['Potential was correct'] = np.NaN
        return row

result = pd.DataFrame(data18.apply(link_values, axis=1))
result = result.dropna(axis='rows')

result = result.sort_values(by=['Name'], ascending=True)

print(result)

correct = result.loc[result['Potential was correct'] == True]
correct_percent = (len(correct.index) / len(result.index)) * 100
print(str("%.2f" % correct_percent) + '% of the potential predictions were correct.')

In [ ]:
print('Do players with age over 30 have a decrement on their overall?')

def link_values(row):

    value18 = row['Overall']

    new_stats = data19.loc[data19['ID'] == row['ID']]
    new_stats = new_stats[['Overall', 'Age']]

    row = pd.Series({'Name': row['Name']})
    if new_stats.size > 0:
        new_stats = new_stats.values[0]
        row['Age'] = new_stats[1]
        row['Overall Difference'] = new_stats[0] - value18
        return row
    else:
        row['Age'] = np.NaN
        row['Overall Difference'] = np.NaN
        return row

result = data18.where(data18['Age'] >= 30)
result = pd.DataFrame(result.apply(link_values, axis=1))
result = result.dropna(axis='rows')

result = result.sort_values(by=['Name'], ascending=True)

had_decrease = result.loc[result['Overall Difference'] < 0]
decrease_percent = (len(had_decrease.index) / len(result.index)) * 100

print(result)
print(str("%.2f" % decrease_percent) + '% of the players over the age of 30 had a decrease in their overall rating.')

In [ ]:
print('The top 10 ranked players of 2018 are the same with the 2019?')
top18 = pd.DataFrame()
top19 = pd.DataFrame()

in_both = []

for i in range(10):
    top18 = top18.append({'Player Name': data18.loc[i]['Name'], 'Overall 2018': data18.loc[i]['Overall']}, ignore_index=True)
    top19 = top19.append({'Player Name': data19.loc[i]['Name'], 'Overall 2019': data19.loc[i]['Overall']}, ignore_index=True)

for i in range(10):
    if top18.iloc[i]['Player Name'] in top19['Player Name'].values:
        in_both.append(top18.iloc[i]['Player Name'])

print(in_both)
print(str(len(in_both)) + ' players appeared on the top 10 charts of both years.')

In [ ]:
print('Does the change of club affect the value and overall rating of a player?')

def link_values(row):

    value18 = row[['Club', 'Overall', 'Value']]

    new_stats = data19.loc[data19['ID'] == row['ID']]
    new_stats = new_stats[['Club', 'Overall', 'Value']]

    row = pd.Series({'Name': row['Name']})
    if new_stats.size > 0:
        new_stats = new_stats.values[0]
        row['Club Changed'] = new_stats[0] != value18['Club']
        row['Overall Increased'] = new_stats[1] > value18['Overall']
        row['Value Increased'] = new_stats[2] > value18['Value']
        return row
    else:
        row['Club Changed'] = np.NaN
        row['Overall Increased'] = np.NaN
        row['Value Increased'] = np.NaN
        return row

result = pd.DataFrame(data18.apply(link_values, axis=1))

result = result.where(result['Club Changed'] == True)
result = result.dropna(axis='rows')
result = result.sort_values(by=['Name'], ascending=True)
print(result)

increased_overall = result.loc[result['Overall Increased'] == True]
increased_value = result.loc[result['Value Increased'] == True]

increased_overall_percent = (len(increased_overall.index) / len(result.index)) * 100
increased_value_percent = (len(increased_value.index) / len(result.index)) * 100
#
print(str("%.2f" % increased_overall_percent) + '% of the players who changed club had an increase in overall rating.')
print(str("%.2f" % increased_value_percent) + '% of the players who changed club had an increase in value.')

In [ ]:
print('All the players that have retired in 2019.')

import time
start = time.time()

# Took 13.61 seconds
result = data18.apply(check_id, axis=1)
result = data18[result].sort_values(by=['Position'], ascending=True)
print(result[['Name', 'Age', 'Position']])

result = result.groupby('Position').mean()
print('List of all positions with the average age of retired players from the position')
print(result['Age'])

# Took 117.3 seconds
# for i in range(MAX):
#     new_stats = data19.loc[data19['ID'] == data18.loc[i, 'ID']]
#     if new_stats.size == 0:
#         result = result.append(data18.iloc[i], ignore_index=True)
# print(result)

end = time.time()
print('time: ')
print(end - start)